# Setup

In [ ]:
import numpy as np

with open('input-small.txt') as f:
    lines = f.readlines()
    lines  = [list(line.strip()) for line in lines]

game_board = np.array(lines)
start = np.where(game_board == '^')[0][0], np.where(game_board == '^')[1][0]


# Part 1

In [ ]:
def move(pos: tuple[int, int], game_board: np.ndarray) -> tuple[tuple[int, int], np.ndarray]:
    y, x = pos
    match game_board[pos]:
        case '^':
            if y == 0:
                game_board[y, x] = 'X'
                return None, game_board
            if game_board[y-1, x] != '#':
                game_board[y-1, x] = '^'
                game_board[y, x] = 'X'
                return (y-1, x), game_board
            else:
                game_board[y, x] = '>'
                return (y, x), game_board
        case '>':
            if x == game_board.shape[1] - 1:
                game_board[y, x] = 'X'
                return None, game_board
            if game_board[y, x+1] != '#':
                game_board[y, x+1] = '>'
                game_board[y, x] = 'X'
                return (y, x+1), game_board
            else:
                game_board[y, x] = 'V'
                return (y, x), game_board
        case 'V':
            if y == game_board.shape[0] - 1:
                game_board[y, x] = 'X'
                return None, game_board
            if game_board[y+1, x] != '#':
                game_board[y+1, x] = 'V'
                game_board[y, x] = 'X'
                return (y+1, x), game_board
            else:
                game_board[y, x] = '<'
                return (y, x), game_board
        case '<':
            if x == 0:
                game_board[y, x] = 'X'
                return None, game_board
            if game_board[y, x-1] != '#':
                game_board[y, x-1] = '<'
                game_board[y, x] = 'X'
                return (y, x-1), game_board
            else:
                game_board[y, x] = '^'
                return (y, x), game_board

pos = start 
game_board_round_1 = game_board.copy()
            
while pos:
    pos, game_board_round_1 = move(pos, game_board_round_1)
    pass

print(np.sum(game_board_round_1 == 'X'))
    

# Part 2

Be ware, on my machine the cell takes ~2m30s to run

In [ ]:
from enum import IntEnum

class Direction(IntEnum):
    UP = 1
    RIGHT = 2
    DOWN = 4
    LEFT = 8
    ANY = 15
    OBSTACLE = 16
    
class CircleException(Exception):
    pass

possible_locations = np.where(game_board_round_1 == 'X')
possible_locations = list(zip(possible_locations[0], possible_locations[1]))

game_board_round_2 = np.ndarray(game_board.shape, dtype=np.int8) 
game_board_round_2[np.where(game_board == '.')] = 0
game_board_round_2[np.where(game_board == '#')] = 16
game_board_round_2[np.where(game_board == '^')] = 0

def move_with_circle_detection(pos: tuple[int, int], direction: Direction, game_board: np.ndarray) -> tuple[tuple[int, int], Direction, np.ndarray]:
    y, x = pos
    if game_board[y, x] & direction:
        raise CircleException
    match direction:
        case Direction.UP:
            if y == 0:
                game_board[y, x] |= Direction.UP
                return None, None, game_board
            if game_board[y-1, x] != Direction.OBSTACLE:
                game_board[y, x] |= Direction.UP
                return (y-1, x), Direction.UP, game_board
            else:
                return (y, x), Direction.RIGHT, game_board
        case Direction.RIGHT:
            if x == game_board.shape[1] - 1:
                game_board[y, x] |= Direction.RIGHT
                return None, None, game_board
            if game_board[y, x+1] != Direction.OBSTACLE:
                game_board[y, x] |= Direction.RIGHT
                return (y, x+1), Direction.RIGHT, game_board
            else:
                return (y, x), Direction.DOWN, game_board
        case Direction.DOWN:
            if y == game_board.shape[0] - 1:
                game_board[y, x] |= Direction.DOWN
                return None, None, game_board
            if game_board[y+1, x] != Direction.OBSTACLE:
                game_board[y, x] |= Direction.DOWN
                return (y+1, x), Direction.DOWN, game_board
            else:
                return (y, x), direction.LEFT, game_board
        case Direction.LEFT:
            if x == 0:
                game_board[y, x] |= Direction.LEFT
                return None, None, game_board
            if game_board[y, x-1] != Direction.OBSTACLE:
                game_board[y, x] |= Direction.LEFT
                return (y, x-1), Direction.LEFT, game_board
            else:
                return (y, x), Direction.UP, game_board
            
            
obstacle_locations_with_loop = 0

for i, obstacle_location in enumerate(possible_locations):
    game_board_in_loop = game_board_round_2.copy()  
    game_board_in_loop[obstacle_location] = Direction.OBSTACLE
    pos = start
    direction = Direction.UP
    while pos:
        try:
            pos, direction, game_board_in_loop = move_with_circle_detection(pos, direction, game_board_in_loop)
        except CircleException:
            print(f"Loop detected when adding obstacle at {obstacle_location}; Possible Location number: {i}")
            obstacle_locations_with_loop += 1
            break
        
print(obstacle_locations_with_loop)